# Thread Pool Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import time

import logging
import threading
import queue

import unittest

from common import testExit

## Generic Worker Thread and Thread Pool

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [2]:
# Default logging level
UNITTEST_LOGGING_LEVEL = logging.WARNING

# Default queue timeout dictates how quickly a worker thread can process the stop event
DEFAULT_QUEUE_TIMEOUT = 1

# Default worker timeout applies to things like socket connections
DEFAULT_WORKER_TIMEOUT = 5

# Default loop interval determines the minimum time between each run of the manager thread 
DEFAULT_MANAGER_INTERVAL = 10

# Default manager threads dictates the number of workers 
DEFAULT_MANAGER_THREADS = 4

# Default status of a Motion
HOST_STATUS_UNKNOWN = 0

In [3]:
class WorkerThread():
    """Generic class for worker threads"""

    def __init__(self, config, logger, motions, stopEvent, inQueue, outQueue, mutex, instance=None):
        """Simple initialisation"""

        self.name = self.__class__.__name__

        self.config = config
        self.logger = logger
        self.motions = motions
        self.stopEvent = stopEvent
        self.inQueue = inQueue
        self.outQueue = outQueue
        self.mutex = mutex
        self.instance = instance

        try: 
            self.timeout = self.config['Threading'][self.name]['Timeout']
        except KeyError:
            self.timeout = DEFAULT_WORKER_TIMEOUT
            
        try: 
            self.queueTimeout = self.config['Queuing']['Timeout']
        except KeyError:
            self.queueTimeout = DEFAULT_QUEUE_TIMEOUT

        self.thread = threading.Thread(target=self.main)


    def start(self):
        """Start the thread"""

        if self.instance:
            self.logger.debug("%s #%d starting", self.name, self.instance)
        else:
            self.logger.debug("%s starting", self.name)

        self.thread.start()


    def main(self):
        """Do the actual processing, until the stop event is detected"""

        # Keep looping until the stop event is detected
        while not self.stopEvent.is_set():
            motion = None

            try:
                motion = self.inQueue.get(block=True, timeout=self.queueTimeout)

                try:
                    self.process(motion)
                    
                except Exception as e:
                    # Setting the Motion status to unknown will ensure a retry, starting with detection
                    motion.status = HOST_STATUS_UNKNOWN
                    
                    # Let the manager thread know the queue item has been consumed
                    self.inQueue.task_done()
                    raise
                    
                else:
                    # Let the manager thread know the queue item has been consumed
                    self.inQueue.task_done()

            except queue.Empty:
                # Everything is fine but check for the stop event before retrying the queue
                pass
            
            except Exception as e:
                self.logger.error("Exception in %s - %s", self.name, e)
                              
        # Ensure the input queue is empty
        self.flush()


    def flush(self):
        """Empty the input queue"""

        # The mutex is just in case the manager thread is populating the input queue
        self.mutex.acquire()

        try:
            # Simple loop to empty the input queue
            while not self.inQueue.empty():
                try:
                    self.inQueue.get_nowait()
                    self.inQueue.task_done()

                except queue.Empty:
                    break

        except Exception as e:
            # Ensure the mutex is always released
            self.mutex.release()
            
            self.logger.error("Exception in %s - %s", self.name, e)
            
        else:
            self.mutex.release()


    def process(self, motion):
        """Default behavior is to warn that the class does not have a process() method"""
        
        self.logger.warning("%s does not have a process() method", self.name)


    def join(self):
        """Join the thread, essentially waiting for it to stop gracefully"""

        self.thread.join()

        if self.instance:
            self.logger.debug("%s #%d finished", self.name, self.instance)
        else:
            self.logger.debug("%s finished", self.name)

In [4]:
class WorkerThreadPool(WorkerThread):
    """Generic class for a worker thread pool, potentially consisting of a manager thread and multiple worker threads"""

    def __init__(self, config, logger, motions, stopEvent, inQueue, outQueue, workerClass):
        """Simple initialisation"""
        
        mutex = threading.Lock()

        super().__init__(config, logger, motions, stopEvent, inQueue, outQueue, mutex)
        
        try:
            self.interval = self.config['Threading'][self.name]['Interval']
        except KeyError:
            self.interval = DEFAULT_MANAGER_INTERVAL

        try: 
            self.threads = self.config['Threading'][self.name]['Threads']
        except KeyError:
            self.threads = DEFAULT_MANAGER_THREADS

        self.workers = []
        for i in range(self.threads):
            worker = workerClass(config, logger, motions, stopEvent, inQueue, outQueue, mutex, instance=i + 1)
            self.workers.append(worker)


    def start(self):
        """Start the worker threads"""

        super().start()
        
        for worker in self.workers:
            worker.start()
        

    def main(self):
        """Do the actual processing, until the stop event is detected"""

        try:
            while not self.stopEvent.is_set():
                # Mutex is required so the workers can safely flush the queue when the stop event is detected
                self.mutex.acquire()

                # The manager must declare how many motions are to be processed
                try:
                    if not self.stopEvent.is_set():
                        motionCount = self.process()

                except Exception:
                    # Ensure the mutex is always released
                    self.mutex.release()
                    
                    # Re-raise the exception so the error is logged
                    raise

                else:
                    # Release the mutex so that it is available to the worker threads
                    self.mutex.release()

                # Wait for the workers to consume the queue
                pc1 = time.perf_counter()
                self.inQueue.join()
                pc2 = time.perf_counter()
                joinTime = pc2 - pc1
                
                # Generate warning if the workers took longer than the specified interval time
                if joinTime > self.interval:
                    self.logger.warning("%s has exceeded the interval time", self.name)

                else:
                    # Ensure the thread pool manager runs no more frequently than the interval time
                    waitTime = self.interval - joinTime
                    self.stopEvent.wait(waitTime)
                
        except Exception as e:
            self.logger.error("Exception in %s - %s", self.name, e)


    def process(self):
        """Default behavior is to warn that the class does not have a process() method"""
        
        self.logger.error("%s does not have a process() method", self.name)
  

    def join(self):
        """Join the worker threads, ensuring they have all stopped cleanly"""
        
        super().join()
        
        for worker in self.workers:
            worker.join()

## Global Variables

A simple counter is used to prove the thread pools are working

In [5]:
mutex = threading.Lock()
total = 0

## Producer Thread Pool

Simple producer thread pool for testing

In [6]:
class ProducerThread(WorkerThread):
    """Thread pool worker to process a single item"""

    def process(self, motion):
        """Do the actual processing of a single item"""

        self.logger.debug("%s is processing motion %s", self.name, motion.identifier)

        self.outQueue.put(motion)

In [7]:
class ProducerThreadPool(WorkerThreadPool):
    """Thread pool manager to produce data items"""

    def __init__(self, config, logger, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, logger, motions, stopEvent, inQueue, outQueue, ProducerThread)
        

    def process(self):
        """The thread pool manager puts items onto the in queue for workers to consume"""
        
        self.logger.debug("%s is queuing data", self.name)
        
        count = 0
        for motion in self.motions:
            self.logger.debug("%s is queuing motion %s", self.name, motion.identifier)
            self.inQueue.put(motion)
            count += 1
                
        return count

## Consumer Thread Pool

Simple consumer thread pool for testing

In [8]:
class ConsumerThread(WorkerThread):
    """Thread pool worker to process a single item"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        global mutex, total
        
        mutex.acquire()
        
        try:
            self.logger.debug("%s is processing motion %s", self.name, motion.identifier)
            total += 1

        except:
            pass

        mutex.release()

In [9]:
class ConsumerThreadPool(WorkerThreadPool):
    """Thread pool manager to inspect motions appearing on the network"""

    def __init__(self, config, logger, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, logger, motions, stopEvent, inQueue, outQueue, ConsumerThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Dummy Motion Class

Simple class to represent a motion

In [10]:
NUM_MOTIONS = 100

In [11]:
class DummyMotion():
    """Dummy class to represent a Motion"""
    
    def __init__(self, identifier):
        '''Only really need a status property'''

        self.identifier = identifier
        self.status = HOST_STATUS_UNKNOWN

## Basic Test

In [12]:
class TestThreadPools(unittest.TestCase):
    '''Class to test thread pools'''

    def getLogger(self):
        """Initialise logging"""

        # Format of log messages
        formatString = "%(asctime)s,%(levelname)s,%(message)s"
        dateTimeFormat = "%Y-%m-%d %H:%M:%S"

        # Create logger
        logger = logging.getLogger('server')
        logger.setLevel(logging.DEBUG)

        # Handle this script being run repeatedly in IDE
        if not logger.hasHandlers():

            # Create console handler which logs using the level specified in server.log
            ch = logging.StreamHandler()
            ch.setLevel(UNITTEST_LOGGING_LEVEL)

            # Create formatter and add it to the handlers
            formatter = logging.Formatter(fmt=formatString, datefmt=dateTimeFormat)
            ch.setFormatter(formatter)

            # Add the handlers to logger
            logger.addHandler(ch)

        return logger


    def createThreadPools(self, config, logger, motions, stopEvent):
        """Create the thread pools, including message queues"""

        threadPools = []

        # Detection   
        producerInQueue = queue.Queue()
        producerOutQueue = queue.Queue()
        producerThreadPool = ProducerThreadPool(config, logger, motions, stopEvent, producerInQueue, producerOutQueue)
        threadPools.append(producerThreadPool)

        # Inspection    
        consumerInQueue = producerOutQueue
        consumerOutQueue = None
        consumerThreadPool = ConsumerThreadPool(config, logger, motions, stopEvent, consumerInQueue, consumerOutQueue)
        threadPools.append(consumerThreadPool)

        return threadPools


    def testThreadPools(self):
        """Test everything in one go"""

        # Read config
        config = {}

        # Initialise logging module
        logger = self.getLogger()

        # Read Motion data
        motions = [DummyMotion(i) for i in range(NUM_MOTIONS)]

        # Global event so that all threads can exit gracefully
        stopEvent = threading.Event()

        # Create the thread pools, including the message queues, etc.
        logger.debug("Creating thread pools")
        threadPools = self.createThreadPools(config, logger, motions, stopEvent)

        # Start up all of the thread pools
        abort = True
        try:
            logger.info("Starting thread pools")

            for threadPool in threadPools:
                threadPool.start()

            logger.info("Thread pools started")

            abort = False

        except Exception:
            logger.error("Thread pools failed to start")

        # Allow 0.5 seconds for all of the threads to do their work
        if not abort:
            time.sleep(0.5)

        # Stop threads gracefully
        logger.info("Stopping thread pools")
        stopEvent.set()
        for threadPool in threadPools:
            threadPool.join()
        logger.info("Thread pools stopped")

        # Test result
        self.assertEqual(total, NUM_MOTIONS)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [13]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 1.007s

OK
